# Prism example

This tutorial demonstrates how to open an existing Speos file, run
the simulation and open the result. After that it explains how to
edit the File and adjust Parameters of the sensor.

## Prerequisites

### Perform imports

In [1]:
import os
from pathlib import Path

from ansys.speos.core import Body, Project, Speos
from ansys.speos.core.kernel.client import (
    default_docker_channel,
)
from ansys.speos.core.launcher import launch_local_speos_rpc_server
from ansys.speos.core.sensor import Sensor3DIrradiance, SensorIrradiance
from ansys.speos.core.simulation import SimulationDirect


### Define constants

The constants help ensure consistency and avoid repetition throughout the example.

In [2]:
HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.

## Model Setup

### Load assets
The assets used to run this example are available in the
[PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.

> **Note:** Make sure you
> have downloaded simulation assets and set ``assets_data_path``
> to point to the assets folder.

In [3]:
if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")

### Start/Connect to Speos RPC Server
This Python client connects to a server where the Speos engine
is running as a service. In this example, the server and
client are the same machine. The launch_local_speos_rpc_method can
be used to start a local instance of the service.

In [4]:
if USE_DOCKER:
    speos = Speos(channel=default_docker_channel())
else:
    speos = launch_local_speos_rpc_server(port=GRPC_PORT)

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/tools/common/cyberchannel.py:188: UserWarning: Starting gRPC client without TLS on localhost:50098. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


## Create project

Load a project from .speos file.

In [5]:
p = Project(speos=speos, path=str(assets_data_path / "Prism.speos" / "Prism.speos"))
print(p)

{
    "name": "Prism",
    "description": "From /app/assets/Prism.speos/Prism.speos",
    "part_guid": "cbec4228-e05f-47b4-9e94-30733770a032",
    "sources": [
        {
            "name": "Surface.1:7758",
            "metadata": {
                "UniqueId": "a80cbe92-2181-410c-89ef-9c6a58670056"
            },
            "source_guid": "b9214e27-2928-4fb3-83e9-853df270e9b3",
            "description": "",
            "source": {
                "name": "Surface.1:7758",
                "surface": {
                    "radiant_flux": {
                        "radiant_value": 8.346636625511586
                    },
                    "intensity_guid": "e1d93c3e-09df-4e85-b1e9-ad5056e8fa0a",
                    "exitance_constant": {
                        "geo_paths": [
                            {
                                "geo_path": "SourceBody:1801986974/face.1:544289899",
                                "reverse_normal": false
                            }
         

/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:811: UserWarning: The pySpeos feature : FaceStub.read_batch needs a Speos Version of 2025 R2 SP0 or higher.
  f_data_list = face_db.read_batch(refs=f_links)
/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:917: UserWarning: The pySpeos feature : SourceSurface needs a Speos Version of 2025 R2 SP0 or higher.
  src_feat = SourceSurface(
/home/runner/work/pyspeos/pyspeos/.venv/lib/python3.13/site-packages/ansys/speos/core/project.py:989: UserWarning: The pySpeos feature : SimulationDirect needs a Speos Version of 2025 R2 SP0 or higher.
  sim_feat = SimulationDirect(


## Preview
This preview method allows you to preview the content of the Speos solver
file.

In [6]:
p.preview()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html lang=&quot;en-US&quot; dir=&quot;ltr&quot;>\n  …

## Retrieve the simulation feature and open result

Run the simulation

In [7]:
sim_features = p.find(name="Prism", feature_type=SimulationDirect)
sim = sim_features[0]
sim.compute_CPU()

[upload_response {
  info {
    uri: "d335e1f1-1ae9-4535-b8f2-a0a50d95b7e2"
    file_name: "Prism.Irradiance.1.xmp"
    file_size: 1568771
  }
  upload_duration {
    nanos: 2903860
  }
}
, upload_response {
  info {
    uri: "f03ebc0e-830c-4773-a73d-cb5ef6c3eabc"
    file_name: "Prism.html"
    file_size: 65519
  }
  upload_duration {
    nanos: 344012
  }
}
]

Use the open_result_image method to review the result

In [8]:
if os.name == "nt":
    from ansys.speos.core.workflow.open_result import open_result_image

    open_result_image(simulation_feature=sim, result_name="Prism.Irradiance.1.xmp")

## Work with sensor

Retrieve the sensor feature.

Modify the sensor setting, e.g. set the spectral type, etc.

In [9]:
irr_features = p.find(name=".*", name_regex=True, feature_type=SensorIrradiance)
irr = irr_features[0]
wl = irr.set_type_spectral().set_wavelengths_range()
wl.start = 500
wl.end = 600
wl.sampling = 11
irr.commit()

Create and add a new sensor, e.g. 3d irradiance sensor

In [10]:
body = p.find(name="PrismBody", name_regex=True, feature_type=Body)[0]
sensor_3d = p.create_sensor(name="3d_irradiance", feature_type=Sensor3DIrradiance)
sensor_3d.geometries = [body.geo_path]
sensor_3d.commit()
sim.set_sensor_paths(["Irradiance.1:564", "3d_irradiance"])
sim.commit()
p.preview()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html lang=&quot;en-US&quot; dir=&quot;ltr&quot;>\n  …

## Re-run the simulation with new sensor definition.

In [11]:
sim.compute_CPU()
if os.name == "nt":
    open_result_image(simulation_feature=sim, result_name="Prism.Irradiance.1.xmp")

In [12]:
speos.close()

True